<div class="h1 alert alert" markdown="1" >

# Ames Housing Data & Kaggle Challenge 
</div>

## Modeling

<img src='../images/houses9.png'>

### You are tasked with creating a machine learning model based on the Ames Housing Dataset. This model will predict the price of a property at sale.
### The Ames Housing Dataset is contains over 70 columns of different features relating to houses.

run me! ↓

In [2]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<font color=white> rpubs.c om/RobbyS/622233 good help (in R)</font>

<font color=white>in R, rly helpful, good markdown explanation github.co m/chenyze/project-2</font>

<div class="alert alert-block alert-info">
From documentation: One issue is in regards to homoscedasticity and assumption violations. The graph included in the article appears to indicate heteroscedasticity with variation increasing with sale price and this problem is evident in many simple home pricing models that focus only on house and lot sizes. Though this violation can be alleviated by transforming the response variable (sale price), the resulting equation yields difficult to interpret fitted values (selling price in log or square root dollars). This situation gives the instructor the opportunity to talk about the costs (biased estimators, incorrect statistical tests, etc.) and benefits (ease of use) of not correcting this assumption violation. If the purpose in building the model is simply to allow a typical buyer or real estate agent to sit down and estimate the selling price of a house, such transformations may be unnecessary or inappropriate for the task at hand. This issue could also open into a discussion on the contrasts and comparisons between data mining, predictive models, and formal statistical inference.
    </div>

<div class="alert alert-block alert-info">
A second issue closely related to the intended use of the model, is the handling of outliers and unusual observations. In general, I instruct my students to never throw away data points simply because they do not match a priori expectations (or other data points). I strongly make this point in the situation where data are being analyzed for research purposes that will be shared with a larger audience. Alternatively, if the purpose is to once again create a common use model to estimate a “typical” sale, it is in the modeler’s best interest to remove any observations that do not seem typical (such as foreclosures or family sales).
    </div>

#obsessed with these maps
https://www.tmwr.org/ames.html

## Heading

### Imports

In [36]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures


pd.set_option('display.max_columns', None)

In [37]:
# Load in data
train = pd.read_csv('../code/train2.csv')
test = pd.read_csv('../code/test2.csv')

In [38]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       878 non-null    int64  
 1   id               878 non-null    int64  
 2   pid              878 non-null    int64  
 3   ms_subclass      878 non-null    int64  
 4   ms_zoning        878 non-null    object 
 5   lot_frontage     718 non-null    float64
 6   lot_area         878 non-null    int64  
 7   street           878 non-null    object 
 8   alley            58 non-null     object 
 9   lot_shape        878 non-null    int64  
 10  land_contour     878 non-null    object 
 11  utilities        878 non-null    int64  
 12  lot_config       878 non-null    object 
 13  land_slope       878 non-null    int64  
 14  neighborhood     878 non-null    object 
 15  condition_1      878 non-null    object 
 16  condition_2      878 non-null    object 
 17  bldg_type       

### Preprocessing

#### Log Tranformation of dependent variable

In [39]:
# Since distribution of price is skewed to the left, I did a log transform 

In [40]:
plt.hist(y_train, bins=20);

NameError: name 'y_train' is not defined

In [41]:
plt.hist(np.log(y_train), bins=20);

NameError: name 'y_train' is not defined

In [42]:
y_train = np.log(y_train)
y_train.head(2)

NameError: name 'y_train' is not defined

In [43]:
y_val = np.log(y_val)
y_val.head(2)

NameError: name 'y_val' is not defined

In [44]:
# log transform before tts - no data leakage issues
# do it to both train and test, or reverse it afterwards

#### make_column_transformer

In [45]:
ohe = OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False)

In [46]:
smart_encoder = make_column_transformer(
    (ohe, ['neighborhood', 'bldg_type']), #can add more
    remainder='drop'      
)

train_ohe = smart_encoder.fit(train)
train_ohe = smart_encoder.transform(train)
train_ohe = pd.DataFrame(train_ohe, columns=smart_encoder.get_feature_names_out())
train_ohe.columns = [x.removeprefix('onehotencoder__') for x in train_ohe.columns]
train_ohe.columns

test_ohe = smart_encoder.transform(test)
test_ohe = pd.DataFrame(test_ohe, columns=smart_encoder.get_feature_names_out())
test_ohe.columns = [x.removeprefix('onehotencoder__') for x in test_ohe.columns]
test_ohe.columns

Index(['neighborhood_Blmngtn', 'neighborhood_Blueste', 'neighborhood_BrDale',
       'neighborhood_BrkSide', 'neighborhood_ClearCr', 'neighborhood_CollgCr',
       'neighborhood_Crawfor', 'neighborhood_Edwards', 'neighborhood_Gilbert',
       'neighborhood_Greens', 'neighborhood_GrnHill', 'neighborhood_IDOTRR',
       'neighborhood_Landmrk', 'neighborhood_MeadowV', 'neighborhood_Mitchel',
       'neighborhood_NAmes', 'neighborhood_NPkVill', 'neighborhood_NWAmes',
       'neighborhood_NoRidge', 'neighborhood_NridgHt', 'neighborhood_OldTown',
       'neighborhood_SWISU', 'neighborhood_Sawyer', 'neighborhood_SawyerW',
       'neighborhood_Somerst', 'neighborhood_StoneBr', 'neighborhood_Timber',
       'neighborhood_Veenker', 'bldg_type_1Fam', 'bldg_type_2fmCon',
       'bldg_type_Duplex', 'bldg_type_Twnhs', 'bldg_type_TwnhsE'],
      dtype='object')

In [47]:
#Make neighborhoods list
neighborhoods = [i for i in train_ohe.iloc[:, :27].columns]
neighborhoods

bldgtypes = [i for i in train_ohe.iloc[:, [28,29,30,31,32]].columns]

In [48]:
# Merge #Don't run me twice
train_ohe = pd.merge(left=train, right=train_ohe, left_index=True, right_index=True)
train_ohe.head(2)

test_ohe = pd.merge(left=test, right=test_ohe, left_index=True, right_index=True)
test_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Columns: 115 entries, Unnamed: 0 to bldg_type_TwnhsE
dtypes: float64(41), int64(53), object(21)
memory usage: 789.0+ KB


In [49]:
# Drop columns : ID, PID, Neighborhood
train_ohe=train_ohe.drop(columns = ['Unnamed: 0', 'neighborhood'], axis=1)
train_ohe.head(2)

test_ohe=test_ohe.drop(columns = ['Unnamed: 0', 'neighborhood'], axis=1)

In [50]:
# Move data cleaning

### Modeling?

In [51]:
# Define X & y

features = ['gr_liv_area',  'overall_qual', 'overall_cond', 'year_built', 'kitchen_qual',       #check nulls
            'garage_area', 'year_remod/add', 'exter_qual', 'exter_cond', 
            'total_bsmt_sf', 'total_sf'] + neighborhoods + bldgtypes
# make total_sf

X = train_ohe[features]         # Fit on training set
y = train['saleprice']        # Fit whatever you're doing on whatever you want the values to be learned from

# flippers will buy high quality things in worse conditions if it's easy to restore
# Condition generally less important than high quality vs builders grade stuff


In [52]:
train_ohe[features].isnull().sum()

gr_liv_area             0
overall_qual            0
overall_cond            0
year_built              0
kitchen_qual            0
garage_area             0
year_remod/add          0
exter_qual              0
exter_cond              0
total_bsmt_sf           0
total_sf                0
neighborhood_Blmngtn    0
neighborhood_Blueste    0
neighborhood_BrDale     0
neighborhood_BrkSide    0
neighborhood_ClearCr    0
neighborhood_CollgCr    0
neighborhood_Crawfor    0
neighborhood_Edwards    0
neighborhood_Gilbert    0
neighborhood_Greens     0
neighborhood_GrnHill    0
neighborhood_IDOTRR     0
neighborhood_Landmrk    0
neighborhood_MeadowV    0
neighborhood_Mitchel    0
neighborhood_NAmes      0
neighborhood_NPkVill    0
neighborhood_NWAmes     0
neighborhood_NoRidge    0
neighborhood_NridgHt    0
neighborhood_OldTown    0
neighborhood_SWISU      0
neighborhood_Sawyer     0
neighborhood_SawyerW    0
neighborhood_Somerst    0
neighborhood_StoneBr    0
neighborhood_Timber     0
bldg_type_1F

In [53]:
train['garage_area'].isnull().sum()

0

#### Train - validation split

In [54]:
# Train test split (Train validation split)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=404)

In [55]:
# cleaning nulls/imputation after split - to avoid data leakage

# mean should be from training set only 

#### Pipeline

In [63]:
# Pipeline can hold [n transformers]+[n estimators] #weird if you have more than 1 est

# Show pipe diagram
from sklearn import set_config
set_config(display='diagram')

# Instantiate pipeline

pipe = make_pipeline(
    # SimpleImputer(),
    # StandardScaler(),
    # PolynomialFeatures(),
    #LinearRegression(),
    Ridge()
)

In [64]:
pipe.named_steps

{'ridge': Ridge()}

In [65]:
# Fit pipe
pipe.fit(X_train, y_train)

Pipeline(steps=[('ridge', Ridge())])

In [66]:
# Evaluate
pipe.score(X_train, y_train), pipe.score(X_val, y_val)


(0.8760687154307417, 0.8880416464822757)

In [70]:
cross_val_score(pipe, X_train, y_train, cv=5).mean()

0.8651948538246321

In [ ]:
Right now am doing different tests and overwriting what I already had
is there a better way to show my progress ?

In [390]:
pipe.named_steps['linearregression'].coef_

array([ 5.23847308e+01,  1.39105643e+04,  2.71204365e+02,  1.23635901e+04,
        4.53525041e+01,  1.13856041e+02,  1.23746864e+04, -2.15079847e+04,
       -3.43286652e+04, -3.42331375e+04, -3.20496226e+03,  2.19671518e+04,
       -1.29389818e+04,  1.52070093e+04, -8.27960970e+03, -1.61297662e+04,
       -6.40274835e+03,  1.18328218e+05, -1.27700399e+04, -1.09139364e-11,
       -2.04207607e+04, -4.19134829e+03, -2.32392574e+03, -2.41752217e+04,
       -5.81912796e+03,  2.08788660e+04,  3.16733256e+04, -1.75914080e+04,
       -1.05464850e+04, -1.15265027e+03, -2.38892580e+04, -1.76793957e+04,
        5.04061123e+04,  5.77054267e+03,  1.33542513e+04])

In [391]:
X_train.head(2)

,gr_liv_area,overall_qual,year_built,kitchen_qual,garage_area,year_remod/add,exter_qual,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker
922,1496,8,2007,4,474.0,2007,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
180,1121,4,2003,3,440.0,2004,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Grid CV

In [340]:
# Hyperparameters dict
pipe_params = {
    '  __ ': [ , ],
    '  __ ': [ , ],
    '  __ ': [ , ]
}

SyntaxError: invalid syntax (1309630455.py, line 3)

In [341]:
# Instantiate GridSearchCV
# Pipeline can be passed into GridCV as an estimator.
gs = GridSearchCV(pipe, param_grid=pipe_params, verbose=1)

NameError: name 'pipe_params' is not defined

In [ ]:
# Fit GridSearchCV to the data
gs.fit(X_train, y_train)

### Test

In [60]:
test['SalePrice'] = pipe.predict(test_ohe[features])


test = test[['id', 'SalePrice']]
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         878 non-null    int64  
 1   SalePrice  878 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 13.8 KB


In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         878 non-null    int64  
 1   SalePrice  878 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 13.8 KB


In [62]:
test.to_csv('predictions4.csv', index=False)

In [396]:
test_ohe.head(7)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,2,Lvl,3,Inside,2,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,2,Stone,2,3,1.0,6.0,0,1.0,0,1020,1020,GasA,4,N,0.0,908,1020,0,1928,0,0,2,0,4,2,2,9,8,0,0,Detchd,1910.0,2.0,1,440,1,1,2,0,60,112,0,0,0,0,0,NaN,0,4,2006,WD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,2,Lvl,3,Inside,2,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,3,3,CBlock,4,3,1.0,6.0,0,1.0,0,1967,1967,GasA,3,Y,0.0,1967,0,0,1967,0,0,2,0,6,2,3,10,8,0,0,Attchd,1977.0,2.0,2,580,3,3,2,170,0,0,0,0,0,0,0,NaN,0,8,2006,WD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,3,Lvl,3,Inside,2,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,3,PConc,4,4,1.0,6.0,554,1.0,0,100,654,GasA,5,Y,0.0,664,832,0,1496,1,0,2,1,3,1,4,7,8,1,4,Attchd,2006.0,1.0,2,426,3,3,2,100,24,0,0,0,0,0,0,NaN,0,9,2006,New,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,3,Lvl,3,Inside,2,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,3,CBlock,3,3,1.0,1.0,0,1.0,0,968,968,GasA,3,Y,0.0,968,0,0,968,0,0,1,0,2,1,3,5,8,0,0,Detchd,1935.0,0.0,2,480,2,3,2,0,0,184,0,0,0,0,0,NaN,0,7,2007,WD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,625,535105100,20,RL,NaN,9500,Pave,NaN,2,Lvl,3,Inside,2,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,3,3,CBlock,4,3,1.0,1.0,609,1.0,0,785,1394,GasA,4,Y,0.0,1394,0,0,1394,1,0,1,1,3,1,3,6,8,2,4,Attchd,1963.0,1.0,2,514,3,3,0,0,76,0,0,185,0,0,0,NaN,0,7,2009,WD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,333,923228370,160,RM,21.0,1890,Pave,NaN,2,Lvl,3,Inside,2,Norm,Norm,TwnhsE,2Story,4,6,1972,1972,Gable,CompShg,CemntBd,CmentBd,None,0.0,3,3,CBlock,3,3,1.0,5.0,294,3.0,0,252,546,GasA,3,Y,0.0,546,546,0,1092,0,0,1,1,3,1,3,5,5,0,0,Attchd,1972.0,0.0,1,286,3,3,2,0,0,64,0,0,0,0,0,NaN,0,6,2010,WD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1327,902427150,20,RM,52.0,8516,Pave,NaN,3,Lvl,3,Inside,2,Norm,Norm,1Fam,1Story,4,6,1958,2006,Gab

In [397]:
preds = lr.predict(X_test)

NameError: name 'lr' is not defined

In [13]:
test.shape

(878, 80)

In [15]:
test['SalePrice'] = preds

In [17]:
submission = test[['Id', 'SalePrice']]

In [18]:
submission.head(3)

,Id,SalePrice
0,2658,228792.198436
1,2718,233097.556248
2,2414,181102.081138


In [19]:
submission.to_csv('basic_model.csv', index = False) #Kaggle won't let you submit if you don't set index.